<a href="https://colab.research.google.com/github/mihirahuja1/Spotify_User_Behavior/blob/master/Compiled_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn import preprocessing
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.express as px

In [2]:
cid= 'CleintID'
secret= 'Secret'

In [3]:

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

In [128]:
username = 'USER_NAME'
pl_id = 'spotify:playlist:PLAYLIST_ID'
personaility = 'UNK'

In [129]:
#sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
#https://open.spotify.com/playlist/37i9dQZF1EpvrG9jgVuHrm?si=lq2F9s-tRVaDAZx-3KGSbA

offset = 0
track_ids = []
while True:
    response = sp.playlist_tracks(pl_id,
                                  offset=offset,
                                  fields='items.track.id,total')
    track_ids.append(response['items'])
    print(response['items'])
    offset = offset + len(response['items'])
    #print(offset, "/", response['total'])

    if len(response['items']) == 0:
        break
def get_track_info(track_uri):
  track_info = sp.track(track_uri)
  artist_name = track_info['album']['artists'][0]['name']
  track_name = track_info['name']
  return track_name
song_df = pd.DataFrame()
list_of_songs = []
for i in range(0,len(track_ids[0])):
  print(track_ids[0][i]['track']['id'])
  list_of_songs.append(track_ids[0][i]['track']['id'])
songs_df = pd.DataFrame(list_of_songs,columns=['Track_URI'])
main_df = pd.DataFrame()
cnt=0

def create_audio_features_df(song):
  global cnt
  global main_df
  d = sp.audio_features(song)[0]
  new_df = pd.DataFrame(d.values())
  new_df = new_df.transpose()
  #print(new_df)
  if cnt == 0:
    main_df = new_df
  else:
    #print('here')
    x = pd.concat([main_df, new_df], ignore_index=True)
    main_df = x
  cnt+=1
  #return main_df
songs_df['Track_URI'].apply(lambda x: create_audio_features_df(x)) 
main_df.columns
main_df.columns = main_df.columns.astype(str)
main_df= main_df[['0','1','2','3','4','5','6','7','8','9','10']]
main_df.columns = ['danceability', 'energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
songs_df = pd.concat([songs_df, main_df], axis=1)
songs_df['Track_Name'] = songs_df['Track_URI'].apply(lambda x: get_track_info(x))
fullpath = "drive/My Drive/SpotifyDS/songs_audio_features_"+str(username) + ".csv"
songs_df.to_csv(fullpath)

[{'track': {'id': '62aP9fBQKYKxi7PDXwcUAS'}}, {'track': {'id': '5v4GgrXPMghOnBBLmveLac'}}, {'track': {'id': '2qzUpSVI4NnPyWxbXwumTj'}}, {'track': {'id': '4HBZA5flZLE435QTztThqH'}}, {'track': {'id': '2A9rFFPwsnhusCh8ZMBvYY'}}, {'track': {'id': '66dQdXAbtuPdSasezCQVZE'}}, {'track': {'id': '3f3cliOygeuUpGRwdohy12'}}, {'track': {'id': '5oZps6mYqU5s7A4WjUZggJ'}}, {'track': {'id': '72794Eag03xdy7TO0KNuid'}}, {'track': {'id': '0EA2RhRHL4KWeNa7JfD1Yw'}}, {'track': {'id': '4CPszpSnlbgnQLIQYyzIR2'}}, {'track': {'id': '3R9RSIkgkZXrIgmBHGemlV'}}, {'track': {'id': '59iNA6gwthoTJ6IgpBGGHW'}}, {'track': {'id': '4gT3mNJA8lnlkYFqGZ8IA2'}}, {'track': {'id': '2Xcvt8NRLw0xbB3ClfW8MI'}}, {'track': {'id': '0Ryd8975WihbObpp5cPW1t'}}, {'track': {'id': '6BvtitRX5lQC87YlA6rq0n'}}, {'track': {'id': '3lB0GMiI5KxDbTOG8V3bOx'}}, {'track': {'id': '62ljuuXo0zrcrtJnnOhfxT'}}, {'track': {'id': '4jAIqgrPjKLTY9Gbez25Qb'}}, {'track': {'id': '017PF4Q3l4DBUiWoXk4OWT'}}, {'track': {'id': '3HAgxyWGeJtIVabS2mTREt'}}, {'track':

# PCA

## Getting information for base user
## Applying normalization and PCA

In [130]:
original_base_user = pd.read_csv('drive/My Drive/SpotifyDS/songs_audio_features.csv')
original_base_audio_information = original_base_user[['danceability',	'energy',	'key','loudness',	'mode','speechiness',	'acousticness',	'instrumentalness',	'liveness',	'valence',	'tempo']]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(original_base_audio_information)
original_base_audio_information = pd.DataFrame(x_scaled)
pca = PCA(n_components=2)#To be run for original scalar
pct = pca.fit_transform(original_base_audio_information)


In [131]:
songs_audio_information = pd.read_csv('drive/My Drive/SpotifyDS/songs_audio_features_USERNAME.csv')

In [132]:
audio_information = songs_audio_information[['danceability',	'energy',	'key','loudness',	'mode','speechiness',	'acousticness',	'instrumentalness',	'liveness',	'valence',	'tempo']]
#min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.transform(audio_information)
audio_information = pd.DataFrame(x_scaled)

In [133]:
pct = pca.transform(audio_information)
principal_df = pd.DataFrame(pct,columns=['pc1','pc2'])
finaldf = pd.concat([principal_df,songs_audio_information['Track_Name']],axis=1)


In [134]:
#df = px.data.iris()
fig = px.scatter(finaldf, x="pc1", y="pc2", color="Track_Name"
                 , hover_data=['Track_Name'])
fig.show()

In [135]:
stored_user_positions = pd.read_csv('drive/My Drive/SpotifyDS/user_position.csv')
#user_position[2:]['Pers']='ENTJ'
user_position = stored_user_positions[['User','X','Y','Pers']]

user_position

,User,X,Y,Pers
0,Test,0.000000e+00,0.000000e+00,UNK
1,Mihir,-3.053113e-17,2.035409e-17,ENTJ
2,Yassine,-6.628938e-02,-2.470566e-01,INFJ
3,anya,-1.728095e-01,-2.087594e-01,ENFP
4,rhea,1.317379e-01,-1.989217e-01,ESFJ
5,Khushmeet,2.809389e-01,1.104892e-01,UNK
6,Anika,9.351143e-02,-4.850136e-02,ENFJ
7,Aanvi,-1.473057e-01,-2.234577e-01,ISFJ


In [136]:
user_position = user_position.append({'User' : username , 'X' : finaldf['pc1'].mean(),'Y':finaldf['pc2'].mean(),'Pers':personaility} , ignore_index=True)

In [137]:
#df = px.data.iris()
fig = px.scatter(user_position, x="X", y="Y", color="Pers"
                 , hover_data=['User'])
fig.show()

In [119]:
user_position.to_csv('drive/My Drive/SpotifyDS/user_position.csv')

In [120]:
user_position

,User,X,Y,Pers
0,Test,0.000000e+00,0.000000e+00,UNK
1,Mihir,-3.053113e-17,2.035409e-17,ENTJ
2,Yassine,-6.628938e-02,-2.470566e-01,INFJ
3,anya,-1.728095e-01,-2.087594e-01,ENFP
4,rhea,1.317379e-01,-1.989217e-01,ESFJ
5,Khushmeet,2.809389e-01,1.104892e-01,UNK
6,Anika,9.351143e-02,-4.850136e-02,ENFJ
7,Aanvi,-1.473057e-01,-2.234577e-01,ISFJ
